In [ ]:
from wholeslidedata.annotation.wholeslideannotation import WholeSlideAnnotation
from wholeslidedata.annotation.types import PointAnnotation as Point, PolygonAnnotation as Polygon
from wholeslidedata.image.wholeslideimage import WholeSlideImage
from matplotlib import pyplot as plt
import numpy as np
import os
from wholeslidedata.annotation import utils as annotation_utils
from wholeslidedata.visualization.plotting import plot_annotations

from cv2 import getAffineTransform, warpAffine

from py.helpers import concat_one, get_outlines, get_patch, ROOT
from py.registration import get_3p_transform

path_to_wsi_he = os.path.join(ROOT, r'annotated\\RASL-04_HE.tiff')
path_to_wsa_he = os.path.join(ROOT, r'annotated\\RASL-04_HE.xml')
path_to_wsi_p53 = os.path.join(ROOT, r'annotated\\RASL-04_P53.tiff')
path_to_wsa_p53 = os.path.join(ROOT, r'annotated\\RASL-04_P53.xml')

In [ ]:
def plot_slide_with_annotation(wsi, wsa, spacing=2.0):
    scale = 1/spacing*0.25

    fig, ax = plt.subplots(1,1)
    slide = wsi.get_slide(spacing)
    ax.imshow(slide)
    plot_annotations(wsa.annotations, ax, title="all annotations", scale=scale)
    plt.show()

In [ ]:
wsa_he = WholeSlideAnnotation(path_to_wsa_he)
wsi_he = WholeSlideImage(path_to_wsi_he)

print(f'label_map: {wsa_he.labels.map}')
print(f'counts per label:  {annotation_utils.get_counts_in_annotations(wsa_he.annotations, wsa_he.labels)}')

In [ ]:
wsa_p53 = WholeSlideAnnotation(path_to_wsa_p53)
wsi_p53 = WholeSlideImage(path_to_wsi_p53)

print(f'label_map: {wsa_p53.labels.map}')
print(f'counts per label:  {annotation_utils.get_counts_in_annotations(wsa_p53.annotations, wsa_p53.labels)}')

In [ ]:
p53_outlines = get_outlines(wsa_p53)
HE_outlines = get_outlines(wsa_he)

spacing = 2.0
scale = 1/spacing*0.25

slide = wsi_he.get_slide(spacing=spacing)
slide_p53 = wsi_p53.get_slide(spacing=spacing)

In [ ]:
from wholeslidedata.visualization.color import get_color

def plot_annotations(
    annotations,
    ax=None,
    color_map=None,
    title="",
    use_base_coordinates=False,
    scale=1.0,
):
    ax = ax or plt

    if use_base_coordinates:
        min_x = min(annotation.bounds[0] for annotation in annotations)
        min_y = min(annotation.bounds[1] for annotation in annotations)
        annotations = [annotation.translate((min_x, min_y)) for annotation in annotations]

    for annotation in annotations:
        color = get_color(annotation, color_map)
        coordinates = np.array(annotation.coordinates) * scale

        if isinstance(annotation, Point):
            coordinates = list(zip(*coordinates))
            ax.scatter(*coordinates, color=color)
        elif isinstance(annotation, Polygon):
            ax.plot(*list(zip(*coordinates)), color=color, linewidth=2)
        else:
            raise ValueError(f"invalid annotation {type(annotation)}")

    if ax == plt:
        plt.axis("equal")
        plt.gca().invert_yaxis()
        plt.show()
    else:
        ax.axis("equal")
        ax.set_title(title)

In [ ]:
"""
After creating a whole slide annotation, we can plot all the annotation within it
"""

plot_slide_with_annotation(wsi_he, wsa_he)
plot_slide_with_annotation(wsi_p53, wsa_p53)

Control points testing

In [ ]:
control_points = np.float32([a.coordinates for a in wsa_he.annotations if isinstance(a, Point)]) * scale
control_points_p53 = np.float32([a.coordinates for a in wsa_p53.annotations if isinstance(a, Point)]) * scale

print(control_points)
print(control_points_p53)

In [ ]:
# Point set registration with numpy
transform = np.linalg.solve(concat_one(control_points_p53[:3]), concat_one(control_points[:3])).T
print(transform)

In [ ]:
# Point set registration with opencv
transform = getAffineTransform(control_points_p53[[0,2,4]], control_points[[0,2,4]])
print(transform)

# The x and y translation should be /32 because the coordinate system is larger for the control points
# The rotation and scale are relative however, so they shouldn't be multiplied

In [ ]:
print(control_points)
print(control_points_p53)
print((transform @ concat_one(control_points_p53).T).T)

In [ ]:
# indentity = np.array([[1.,0.,0.],[0.,1.,0.]])
manual_transform = np.array([[1.,0.,-3600.],[0.,1.,-31.]])
# slide_p53_prime = warpAffine(slide_p53, transform, slide.shape[:2][::-1])
# slide_p53_prime = warpAffine(slide_p53, indentity, slide.shape[:2][::-1])
slide_p53_prime = warpAffine(slide_p53, transform, slide.shape[:2][::-1])

In [ ]:
print(transform)
print(manual_transform)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,10))

axes[0].imshow(slide)

# plot_annotations(annotations_p53, axes[1], title='all annotations', scale=1/32)
print("control points HE\n", control_points)
print("control points p53\n", np.array([concat_one(point[None,:])[0] for point in control_points_p53]))
control_points_p53_prime = np.array([(transform @ concat_one(point[None,:]).T).T[0] for point in control_points_p53])
print("transformed control points p53\n", control_points_p53_prime)

axes[0].scatter(control_points[:,0], control_points[:,1], s=100)
axes[0].scatter(control_points_p53_prime[:,0], control_points_p53_prime[:,1])

axes[1].imshow(slide_p53_prime)

axes[1].scatter(control_points[:,0], control_points[:,1], s=100)
axes[1].scatter(control_points_p53_prime[:,0], control_points_p53_prime[:,1])
axes[1].scatter(control_points_p53[[0,2,4],0], control_points_p53[[0,2,4],1])

for i, p in enumerate(control_points_p53):
    axes[1].annotate(i, (p[0], p[1]))

plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,10))

ax.imshow(slide)
ax.imshow(slide_p53_prime, alpha=0.7)

ax.scatter(control_points[:,0], control_points[:,1], s=100)
ax.scatter(control_points_p53_prime[:,0], control_points_p53_prime[:,1])

plt.show()

In [ ]:
transform = get_3p_transform(wsa_p53.annotations, wsa_he.annotations, spacing, [0,2,4])
print(transform)
scale = 1/spacing*0.25

HE_outline = (transform @ concat_one(p53_outlines[0]*scale).T).T / scale

biopsy_img1 = get_patch(wsi_p53, p53_outlines[0], spacing)
biopsy_img2 = get_patch(wsi_he, HE_outline, spacing)

fig, ax = plt.subplots(3,1,figsize=(15,15))
ax[0].imshow(biopsy_img1)
ax[2].imshow(biopsy_img1)

ax[1].imshow(biopsy_img2)
ax[2].imshow(biopsy_img2, alpha=0.5)